In [156]:
from truss_torch_new import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

joints = [
    2, # 0 PIN
    1, # 1 GUSSET
    1, # 2 GUSSET
    1, # 3 GUSSET
    3, # 4 ROLLER
    1, # 5 GUSSET
    1, # 6 GUSSET
    1, # 7 GUSSET
    1, # 8 GUSSET
    1  # 9 GUSSET
    ]

links = [[0, 1], [1,2], [2,3], [3,4], [0,5], [1,5], [1,6], [2,6], [2,7], [2,8], [3,8], [3,9], [4,9], [5,6], [6,7], [7,8], [8,9]]

FORCE_PER_M = 2.5
bridge_joints = [0, 1, 2, 3, 4]

truss = Truss(joints, links, bridge_joints, DEVICE)

{0: [(1, 0), (5, 4)], 1: [(0, 0), (2, 1), (5, 5), (6, 6)], 2: [(1, 1), (3, 2), (6, 7), (7, 8), (8, 9)], 3: [(2, 2), (4, 3), (8, 10), (9, 11)], 4: [(3, 3), (9, 12)], 5: [(0, 4), (1, 5), (6, 13)], 6: [(1, 6), (2, 7), (5, 13), (7, 14)], 7: [(2, 8), (6, 14), (8, 15)], 8: [(2, 9), (3, 10), (7, 15), (9, 16)], 9: [(3, 11), (4, 12), (8, 16)]}


In [157]:
test_input = torch.tensor([[[0, 0], [3.5, 0], [7, 0], [10.5, 0], [14, 0], [-6.6+7, -1.9], [-4.1+7, -4], [7, -4.9], [4.9+7, -4], [6.4+7, -1.9]]], requires_grad=True).to(DEVICE)
test_new = torch.tensor([[[0, 0], [1, 0], [0, 0], [0, 0], [0, 0], [0, -1.9], [0, -4], [0, 0], [0, 0], [0, 0]]], requires_grad=True).to(DEVICE)

batch_test_input = test_input.tile(5,1,1)

X, cost= truss.solve(batch_test_input)

print(cost)



tensor([1676.1561, 1676.1561, 1676.1561, 1676.1561, 1676.1561],
       device='cuda:0', grad_fn=<AddBackward0>)


In [158]:
class OptimizerNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        layers = [
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        ]
        
        for layer in layers:
            if type(layer) == nn.Linear:
                nn.init.kaiming_normal_(layer.weight)

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)
    
def inequalities(X, w0=1, w1=1):
    loss = 0
    relu = nn.ReLU()
    for j in range(len(bridge_joints)-1):
        j0 = bridge_joints[j]
        j1 = bridge_joints[j+1]
        # j1 cannot be further from j0 by 3.5m
        other0 = X[j0]
        other1 = X[j1]
        # if j == 0:
            # other0 = 0
        # if j == len(bridge_joints)-2:
            # other1 = 14
        loss += relu(other1-other0-3.5)*w0
    loss += abs(X[0])
    # for j0 in range(len(joints)):
    #     for j1 in range(len(joints)):
    #         if(j1 is not j0):
    #             # joints cannot be closer than 1m
    #             other0 = X[j0].clone()
    #             if j0 in bridge_joints:
    #                 other0 = 0
    #             other1 = X[j1].clone()
    #             if j1 in bridge_joints:
    #                 other1[...,1] = 0
    #             loss += relu(-truss.length(other0, other1)+1.0)*w1
    return torch.sum(loss)

# print(inequalities(test_input))
known_indices = [0, 1, 3, 5, 7, 8, 9]
knowns = torch.tensor([0.0, 0.0, 0.0, 0.0, 0.0, 14.0, 0.0]).to(DEVICE)

In [159]:
# input = torch.tensor([[[0, 0], [4.5, 0], [7, 0], [10.5, 0], [14, 0], [-6.6+7, -1.9], [-4.1+7, -4], [7, -4.9], [4.9+7, -4], [6.4+7, -1.9]]], requires_grad=True).to(DEVICE)
input = torch.tensor([4.5, 7, 10.5, -6.6+7, -1.9, -4.1+7, -4, 7, -4.9, 4.9+7, -4, 6.4+7, -1.9])
knowns = torch.tensor([0, 4.5, 7, 10.5, 14, -6.6+7, -4.1+7, 7, 4.9+7, 6.4+7, 0, 0, 0, 0, 0, -1.9, -4, -4.9, -4, -1.9])
# input = torch.rand(1, 10, 2).to(DEVICE) * 28 - 14

param = torch.nn.parameter.Parameter(knowns)
optimizer = torch.optim.SGD([param], lr=0.001)

for i in range(10000):
    loss = inequalities(param)
    loss.backward()
    optimizer.step()
    print(loss, param.grad)
    if loss == 0:
        break;

print(param)


tensor(1., grad_fn=<SumBackward0>) tensor([-1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])
tensor(0.9990, grad_fn=<SumBackward0>) tensor([-1.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])
tensor(0.9970, grad_fn=<SumBackward0>) tensor([-1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])
tensor(0.9940, grad_fn=<SumBackward0>) tensor([-1.,  4.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])
tensor(0.9900, grad_fn=<SumBackward0>) tensor([-1.,  5.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])
tensor(0.9850, grad_fn=<SumBackward0>) tensor([-1.,  6.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.])
tensor(0.9790, grad_fn=<SumBackward0>) tensor([-1.,  7.,  0.,  0.,  

In [160]:
# net = OptimizerNet(20, 256, 20)

# net.to(DEVICE)
# optim = torch.optim.Adam(net.parameters(), lr=0.001)

# batch = 1
# batch_test_input = test_input.tile(batch,1,1)
# loss_fn = torch.nn.MSELoss(reduction='sum')

# epochs = 10000
# lowest = 100000
# lowest_success = 100000
# for i in range(epochs):
#     net.train()
#     optim.zero_grad()

#     X = net(batch_test_input.flatten(start_dim=1))
#     X[:, known_indices] = knowns
#     X = X.reshape(batch, 10, 2)

#     sol, cost = truss.solve(X)
#     # loss = loss_fn(cost, torch.ones(batch, 1).to(DEVICE)) + inequalities(X)
#     loss = inequalities(X)
#     loss.backward()
#     optim.step()
#     net.eval()
#     X_test = net(test_input.flatten(start_dim=1))
#     X_test[:, known_indices] = knowns
#     X_test = X_test.reshape(1, 10, 2)

#     sol_test, cost_test = truss.solve(X_test)
#     lowest = min(lowest, float(cost_test))
#     success = float(inequalities(X_test)) == 0
#     if(success):
#         lowest_success = min(lowest_success, float(cost_test))
#     print(float(cost_test), lowest, lowest_success, success, float(inequalities(X_test)))
    